In [23]:
import datetime
import pandas as pd
import requests as re
from pprint import pprint as pp
import threading
import time
import sqlite3

In [24]:
harry = pd.read_csv("sbin.csv").iloc[-1]
# df = pd.DataFrame(harry)
data = harry
data

Timestamp      2023-03-17T15:30:00+0530
Open                              531.0
High                              531.0
Low                               531.0
Close                             531.0
Volume                              580
tr0                                 0.0
tr1                                 0.0
tr2                                 0.0
TR                                  0.0
ATR                             0.65172
BUB                               532.3
BLB                               529.7
FUB                              531.46
FLB                              530.16
ST                               530.16
ST_BUY_SELL                         BUY
Name: 1125, dtype: object

In [26]:
newrow = data.to_json()
# newrow = newrow[newrow[len-1]];
newrow

'{"Timestamp":"2023-03-17T15:30:00+0530","Open":531.0,"High":531.0,"Low":531.0,"Close":531.0,"Volume":580,"tr0":0.0,"tr1":0.0,"tr2":0.0,"TR":0.0,"ATR":0.6517195944,"BUB":532.3,"BLB":529.7,"FUB":531.46,"FLB":530.16,"ST":530.16,"ST_BUY_SELL":"BUY"}'

,Timestamp,Open,High,Low,Close,Volume,tr0,tr1,tr2,TR,ATR,BUB,BLB,FUB,FLB,ST,ST_BUY_SELL
0,2023-03-15T09:15:00+0530,529.80,531.85,529.60,530.90,253865,2.25,NaN,NaN,2.25,0.000000,530.72,530.72,0.00,0.00,0.00,NaN
1,2023-03-15T09:16:00+0530,530.95,531.30,529.80,530.50,122662,1.50,0.40,1.10,1.50,0.107143,530.76,530.34,530.76,530.34,530.76,SELL
2,2023-03-15T09:17:00+0530,530.20,530.50,529.05,529.60,117946,1.45,0.00,1.45,1.45,0.203061,530.18,529.37,530.18,530.34,530.18,SELL
3,2023-03-15T09:18:00+0530,529.70,529.80,529.10,529.65,84080,0.70,0.20,0.50,0.70,0.238557,529.93,528.97,529.93,528.97,529.93,SELL
4,2023-03-15T09:19:00+0530,529.65,530.00,529.05,529.20,87324,0.95,0.35,0.60,0.95,0.289374,530.10,528.95,529.93,528.97,529.93,SELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,2023-03-17T15:26:00+0530,531.45,531.60,531.35,531.45,349090,0.25,0.20,0.05,0.25,0.658139,532.79,530.16,531.46,530.16,530.16,BUY
1122,2023-03-17T15:27:00+0530,531.35,531.50,531.05,531.35,177466,0.45,0.05,0.40,0.45,0.643272,532.56,529.99,531.46,530.16,530.16,BUY
1123,2023-03-17T15:28:00+0530,531.05,531.30,530.80,531.00,73802,0.50,0.05,0.55,0.55,0.636610,532.32,529.78,531.46,530.16,530.16,BUY
1124,2023-03-17T15:29:00+0530,531.00,531.40,529.85,531.00,85647,1.55,0.40,1.15,1.55,0.701852,532.03,529.22,531.46,530.16,530.16,BUY


In [ ]:
def SuperTrend(newrow,data, period = 10, multiplier=3, ohlc=['Open', 'High', 'Low', 'Close']):
        global supertrend
        tr0 = abs(newrow[2] - newrow[3])
        tr1 = abs(newrow[2] - data["close"])
        tr2 = abs(newrow[3]- data["close"])
        TR = round(max([tr0, tr1, tr2]),2)
        
        ST=data["ST"]
        ATR = (data['ATR'] * (period-1))+TR/period
        BUB = round(((newrow[2] + newrow[3]) / 2) + (multiplier * ATR),2)
        BLB = round(((newrow[2] + newrow[3]) / 2) - (multiplier * ATR),2)
        
        if (BUB<data["final_ub"])|(data["close"]>data["final_ub"]):
            final_ub= BUB
        else:
            final_ub=data["final_ub"]

        if (BLB>data["final_lb"])|(data["close"]<data["final_lb"]):
            final_lb=BLB
        else:
            final_lb=data["final_lb"]
            
            
       
        if (ST==data['final_ub']) & (newrow[4]<=final_ub):
            ST =final_ub
        elif (data["ST"]==data["final_ub"])&(newrow[4]>final_ub):
            ST=final_lb
        elif (data["ST"]==data["final_lb"])&(newrow[4]>=final_lb):
            ST=final_lb
        elif (data["ST"]==data["final_lb"])&(newrow[4]<final_lb):
            ST=final_ub
            
    
        if (ST<data["close"]) :
            STX="green"
        else:
            STX="red"
            
        newrow_list = {"Timestamp":newrow[0],"open":newrow[1],"high":newrow[2],"low":newrow[3],"close":newrow[4],"volume":newrow[5],"ATR":ATR,"final_ub":final_ub,"final_lb":final_lb,"ST":ST, "STX":STX}
        
        
        return newrow_list

In [ ]:
def SuperTrendd(data, period = 10, multiplier=3, ohlc=['open', 'high', 'low', 'close']):
        global supertrend
        data['tr0'] = abs(data["high"] - data["low"])
        data['tr1'] = abs(data["high"] - data["close"].shift(1))
        data['tr2'] = abs(data["low"]- data["close"].shift(1))
        data["TR"] = round(data[['tr0', 'tr1', 'tr2']].max(axis=1),2)
        data["ATR"]=0.00
        data['BUB']=0.00
        data["BLB"]=0.00
        data["final_ub"]=0.00
        data["final_lb"]=0.00
        data["ST"]=0.00
        for i, row in data.iterrows():
            if i == 0:
                data.loc[i,'ATR'] = 0.00#data['ATR'].iat[0]
            else:
                data.loc[i,'ATR'] = ((data.loc[i-1,'ATR'] * (period-1))+data.loc[i,'TR'])/period
        data['BUB'] = round(((data["high"] + data["low"]) / 2) + (multiplier * data["ATR"]),2)
        data['BLB'] = round(((data["high"] + data["low"]) / 2) - (multiplier * data["ATR"]),2)
        
        if (data.loc[i,"BUB"]<data.loc[i-1,"final_ub"])|(data.loc[i-1,"close"]>data.loc[i-1,"final_ub"]):
            data.loc[i,"final_ub"]=data.loc[i,"BUB"]
        else:
            data.loc[i,"final_ub"]=data.loc[i-1,"final_ub"]

        if (data.loc[i,"BLB"]>data.loc[i-1,"final_lb"])|(data.loc[i-1,"close"]<data.loc[i-1,"final_lb"]):
            data.loc[i,"final_lb"]=data.loc[i,"BLB"]
        else:
            data.loc[i,"final_lb"]=data.loc[i-1,"final_lb"]

        for i, row in data.iterrows():
            if i==0:
                data.loc[i,"ST"]=0.00
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"]) & (data.loc[i,"close"]<=data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_ub"])&(data.loc[i,"close"]>data.loc[i,"final_ub"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]>=data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_lb"]
            elif (data.loc[i-1,"ST"]==data.loc[i-1,"final_lb"])&(data.loc[i,"close"]<data.loc[i,"final_lb"]):
                data.loc[i,"ST"]=data.loc[i,"final_ub"]

        # Buy Sell Indicator
        for i, row in data.iterrows():
            if i==0:
                data["STX"]="NA"
            elif (data.loc[i,"ST"]<data.loc[i,"close"]) :
                data.loc[i,"STX"]="green"
            else:
                data.loc[i,"STX"]="red"
        return data